In [2]:
import pandas as pd
import requests
import random

In [3]:
url='https://fsp-draw-service.uefa.com/v1/draws?drawId=ecd5e41e-a90c-481d-b7f6-6706be7900c5&optionalFields=TEAMS'

def read_url(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    r=requests.get(url, headers=headers)
    # r.encoding = 'utf-8' 
    if r.ok:
        return r
    
data=read_url(url)
data

<Response [200]>

In [4]:
# Team info
dict_team={}
dict_team={team['teams'][0]['id']: 
           {
            'teamCode': team['teams'][0]['teamCode'], 
            'name': team['teams'][0]['internationalName']
            }
           for team in data.json()[0]['rounds'][-1]['result']['groups'][-1]['teamSlots']}

# pot info
dict_pots={pot['description']: pot['teamIds'] for pot in data.json()[0]['rounds'][0]['pots']}

# match by team
dict_match={}
for team in data.json()[0]['rounds'][0]['result']['groups'][0]['teamSlots']:
    dict_match[team['teamIds'][0]]=[{'opponent': i['teamId'], 'location': i['location']}
                     for i in team['opponents']]
    
# list of total match (first is home team, second is away team)
lst_match=[]
for team, opponents in dict_match.items():
    for opponent in opponents:
        if opponent['location']=='HOME':
            home=team
            away=opponent['opponent']
        else:
            away=team
            home=opponent['opponent']
        lst_match.append((home, away))
lst_match=list(dict.fromkeys(lst_match))

In [78]:

def math_result(home_team, away_team):
    home_pot=[key for key, value in dict_pots.items() if home_team in value][0]
    away_pot=[key for key, value in dict_pots.items() if away_team in value][0]
    dict_ability={'pot 1': 6, 'pot 2': 5, 'pot 3': 4, 'pot 4': 3}

    rate=random.random()
    home_ability=dict_ability[home_pot]
    away_ability=dict_ability[away_pot]
    base_rate=home_ability/(home_ability+away_ability)
    if base_rate<0.5:
        home_win_rate=base_rate*(1-1/3)
        away_win_rate=1-home_win_rate-(1/3)
    else:
        away_win_rate=(1-base_rate)*(1-1/3)
        home_win_rate=1-away_win_rate-(1/3)
    if rate<=min(home_win_rate, away_win_rate):
        if home_win_rate<away_win_rate:
            result=(3, 0)
        else:
            result=(0, 3)
    elif rate<=max(home_win_rate, away_win_rate):
        result=(1, 1)
    else:
        if home_win_rate<away_win_rate:
            result=(0, 3)
        else:
            result=(3, 0)
    return result


home_team='50037'
away_team='52797'
home_pot=[key for key, value in dict_pots.items() if home_team in value][0]
away_pot=[key for key, value in dict_pots.items() if away_team in value][0]
dict_ability={'pot 1': 7, 'pot 2': 5, 'pot 3': 3, 'pot 4': 1}

home_ability=dict_ability[home_pot]
away_ability=dict_ability[away_pot]
base_rate=home_ability/(home_ability+away_ability)
home_win_rate=base_rate*(1-1/3)
away_win_rate=1-home_win_rate-(1/3)
print(dict_team[home_team], dict_team[away_team])
print(base_rate, home_win_rate, away_win_rate)

math_result(home_team, away_team)


{'teamCode': 'BAY', 'name': 'Bayern München'} {'teamCode': 'SLB', 'name': 'S. Bratislava'}
0.875 0.5833333333333334 0.08333333333333331


(3, 0)

In [76]:
for teams, points in zip(lst_match, lst_result):
    if '52797' in teams:
        print(teams, dict_team[teams[0]]['name'], dict_team[teams[1]]['name'], points)

('52797', '52919') S. Bratislava Man City (1, 1)
('52797', '50058') S. Bratislava Milan (0, 3)
('52797', '50164') S. Bratislava GNK Dinamo (3, 0)
('52797', '50107') S. Bratislava Stuttgart (3, 0)
('50037', '52797') Bayern München S. Bratislava (0, 3)
('50124', '52797') Atleti S. Bratislava (0, 3)
('50050', '52797') Celtic S. Bratislava (0, 3)
('2602798', '52797') Girona S. Bratislava (0, 3)


In [82]:
def simulate_result():
    lst_result=[]
    for match in lst_match:
        home_team=match[0]
        away_team=match[1]
        lst_result.append(math_result(home_team, away_team))

    # initialize dict_table
    dict_table={team: 0 for team in dict_team.keys()}

    for teams, points in zip(lst_match, lst_result):
        dict_table[teams[0]]=dict_table[teams[0]]+points[0]
        dict_table[teams[1]]=dict_table[teams[1]]+points[1]

    df=pd.DataFrame(dict_table, index=['result']).transpose()
    df=df.sort_values('result', ascending=False)
    df=df.reset_index()
    df['Name']=df['index'].apply(lambda i: dict_team[i]['name'])
    return df


In [89]:
for i in range(10):
    df_result=simulate_result()
    print(df_result[df_result['Name']=='Liverpool']['result'])

8    14
Name: result, dtype: int64
17    12
Name: result, dtype: int64
2    18
Name: result, dtype: int64
11    15
Name: result, dtype: int64
21    10
Name: result, dtype: int64
7    14
Name: result, dtype: int64
30    7
Name: result, dtype: int64
17    12
Name: result, dtype: int64
34    6
Name: result, dtype: int64
24    10
Name: result, dtype: int64
